<a href="https://colab.research.google.com/github/megan-the-astronomer/ASTR229/blob/main/estimating_exposure_times.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How many photons do you expect?

In a perfect world, our astronomical images would contain signal from our objects - and only signal from our objects. In practice, this is never the case because there will be some signal from the background sky around the object and from the detector itself. As we will discuss in class, all these things contribute to the noise in an image.

When designing observations, we want to optimize the signal-to-noise ratio ($S/N$ sometimes written as SNR although this can be confused with the abbreviation for supernova remnant). The signal-to-noise equation for a CCD detector is

\begin{equation}
\frac{S_o}{N} = \frac{S_o}{\sqrt{S_o + S_s + S_d + R^2}}
\end{equation}

where
$S_o$ is the signal from the source
$S_s$ is the signal from the background / sky,
$S_d$ is the signal from the dark current
and
$R$ is the readnoise.

For this project, we want our observations to be object limited, that is
$S_o \gg S_s, S_d, R^2$.
In this regime, the biggest source of uncertainty is photon noise from the object, so we can approximate the signal-to-noise ratio as $S/N \sim \sqrt{S_o}$.

We'll use this to determine how long we need to observe our targets to get a good image. Our goal is to obtain a $S/N \sim 100$.

In [ ]:
# start by importing a few essentials
import numpy as np
import matplotlib.pyplot as plt
import astropy

from astropy.io import fits
from astropy.table import Table
import astropy.constants as const
import astropy.units as u

The signal, $S_o$, is the expected number of photons from our object. If we derive an estimated photon flux, that is the number of photons / second, then we can work out the required exposure time to reach our desired $S/N$.

We can do this using a magnitude or flux measurement and a little math and physics.

First, we know that the **flux**, $F$, has units of
*energy / area / time / wavelength*.  

Next, we need to know the **wavelength** that we want to observe.
We can use this to estimate the average energy per photon for a given wavelength.
This also tells us the bandpass, or how big our bucket is in wavelength space.

To determine the **collecting area**, we have to know the size of the telescope. We'll also make an approximation for atmospheric and telescope losses.

We'll need to think about how light is distributed on the detector to determine how many pixels our object will cover.

Finally, we'll estimate the exposure time required to achieve $S/N \sim 100$.

As an example, consider a star with $V=15$ mag which has a flux $F = 3.62 \times 10^{-15}$ erg cm$^{-2}$ s$^{-1}$ Å$^{-1}$.

In [ ]:
ref_mag = 15
ref_flux = 3.62e-15 * u.erg / u.s / u.cm**2 / u.Angstrom

In [ ]:
ref_flux

<Quantity 3.62e-15 erg / (Angstrom s cm2)>

# Wavelengths: the effective wavelength and the bandpass

Use the filter transmission data from ```PyAstronomy``` that you used in the "making models" assignment to calculate the effective wavelength and the bandpass of the filter.

The effective wavelength is the wavelength where the transmission is optimized. You can compute this as the weighted average of the wavelengths in the bandpass.

The filter bandpass is the range of wavelengths transmitted by a given filter and is usually given by the full-width-half-maximum of the transimission curve.

In [ ]:
pip install PyAstronomy

In [ ]:
# get the astronomy library
from PyAstronomy import pyasl

In [ ]:
tcs = pyasl.TransmissionCurves()

In [ ]:
vfilter = tcs.getTransCurveData('Bessel v')

In [ ]:
# extract the wavelengths for the filter
wvs = vfilter[:,0]

# extract the transmission at each wavelength
trns = vfilter[:,1]

# give each wavelength a relative weight based on its transmission
weights = wvs*trns

# calculate the effective wavelength of the filter (Angstroms)
eff_wav = np.trapz(weights, x=wvs) / np.trapz(trns, x=wvs)

In [ ]:
eff_wav

5512.103907737096

Let's keep track of our units to make our lives easier later.

In [ ]:
eff_wav = eff_wav * u.Angstrom

The bandwidth is just the width of the filter in wavelength units. We'll use the approximate FWHM of the filter to represent the bandwidth.

In [ ]:
w = np.where(trns > 0.45)

In [ ]:
wvs[w].min()

5000.0

In [ ]:
wvs[w].max()

5900.0

In [ ]:
bandpass = (wvs[w].max() - wvs[w].min()) * u.Angstrom

In [ ]:
bandpass

<Quantity 900. Angstrom>

# Energy

The energy of a photon depends on the wavelength, $\lambda$ as

\begin{equation}
E_{photon} = h \nu = \frac{hc}{\lambda}
\end{equation}

where $h$ is Planck's constant and $c$ is the speed of light.  

Using the effective wavelength that we just calculated, we can estimate the typical energy of a photon in the band.

In [ ]:
E_V_photon = const.h * const.c / eff_wav

In [ ]:
E_V_photon

<Quantity 3.60378884e-29 J m / Angstrom>

Notice the weird units. We can fix this using the unit conversion feature of the ```units``` package.

In [ ]:
E_V_photon.to(u.erg)

<Quantity 3.60378884e-12 erg>

In [ ]:
# note this is energy per photon!!
E_V_photon = E_V_photon.to(u.erg)

With the average energy per photon, we can estimate the average photon flux.

In [ ]:
photon_flux = ref_flux / E_V_photon

In [ ]:
photon_flux

<Quantity 0.0010045 1 / (Angstrom s cm2)>

So we can expect roughly $1 \times 10^3$ photons cm$^{-2}$ s$^{-1}$ Å$^{-1}$.

Including the bandpass, we find the number of photons cm$^{-2}$ s$^{-1}$.

In [ ]:
photon_flux *= bandpass

Notice the syntax - we're taking the value that was assigned to ```photon_flux``` multiplying it by ```bandpass``` and then assigning the result to ```photon_flux```. This makes for very compact code but it can be easy to make mistakes, especially when re-running cells. Buyer beware.

In [ ]:
photon_flux

<Quantity 0.90404853 1 / (s cm2)>

# Area

The area term is just the collecting area of the telescope. Easy.

\begin{equation}
A = \pi r^2 = \pi \left( \frac{D}{2} \right)^2
\end{equation}

In [ ]:
McD_D = 0.8 *u.m
McD_area = np.pi * (McD_D/2.)**2

In [ ]:
McD_area

<Quantity 0.50265482 m2>

Use the collecting area to estimate the photons per second.

In [ ]:
photon_flux *= McD_area.to(u.cm**2)

In [ ]:
photon_flux

<Quantity 4544.24354257 1 / s>

# Estimating telescope losses

In an ideal world, we would detect photons with perfect efficiency. In practice, we lose photons to the atmosphere, the telescope optics, imperfect filter transmission, the response of the detector, ...

For a given observatory and/or an instrument, a lot of careful work goes into characterizing the losses to each of these components. For this exercise, we'll make the extremely simple assumption that we only detect 50\% of the incoming photons.

Does this sound pessimistic to you? Take a look at some of the throughput curves for the [Advanced Camera for Surveys (ACS) on Hubble](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/system-throughputs) and [NIRCam on Webb](https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-instrumentation/nircam-filters#gsc.tab=0).
Remember, both of the these observatories are in space so these curves do not include atmospheric losses!

In [ ]:
photon_flux *= 0.5

In [ ]:
photon_flux

<Quantity 2272.12177129 1 / s>

# Seeing

As we have discussed in class, the seeing spreads out the light from a star. In practice, this means that light from your object will be spread over more or fewer pixels depending on how good the seeing is.

We want to know roughly how many photons we will get in a given pixel so we need to know two things:

(1) the typical seeing

and

(2) the pixel size.

Assume that the typical seeing at McDonald is $\sim 2.5^{\prime\prime}$.

Use the unbinned pixel size ($1.3553^{\prime\prime}$/pix) to calculate how many pixels the light from your star is spread over.

In [ ]:
seeing = 2.5 * u.arcsec
pix_scale = 1.3553 * u.arcsec / u.pix

In [ ]:
# the seeing disk is round
area_seeing_disk = np.pi * (seeing / 2.)**2

# pixels are square
area_pixel = pix_scale**2

In [ ]:
n_pix = area_seeing_disk / area_pixel

In [ ]:
n_pix

<Quantity 2.67238498 pix2>

The photons per second we calculated will be spread out over this area.

In [ ]:
photon_flux / n_pix

<Quantity 850.22247499 1 / (s pix2)>

# Turning this into an exposure time

Now you have a expected number of photons / second / pixel (now is a great time to check your units if you haven't already).

For our imaging projects, we want photon noise from the source to be the dominant source of uncertainty so that $S/N \sim \sqrt{S_o}$ where $S_o$ is the signal from your object.

Use this to estimate how long you would need to observe your target to reach $S/N \sim 100$.   

In [ ]:
# code

# How can I use this to calculate the exposure time for *my* objects??

Some things to keep in mind.

We've talked about magnitudes as they relate to flux,
\begin{equation}
m = -2.5 \log{F} + K
\end{equation}

where
$m$ is the apparent magnitude,
$F$ is the flux, and
$K$ is a constant to put the magnitude into a given system. Curious to know more about magnitude systems? Take a look at Chapter 10 in Chromey.
If we want to put our observations on a standard scale, we'll need to know $K$.

We can also measure the apparent magnitude relative to another object with a known brightness using

\begin{equation}
m_1 - m_2 = -2.5 \log{\frac{F_1}{F_2}}
\end{equation}

where the subscripts $1$ and $2$ reflect the two sources.

In [ ]:
# your code

# But I want to observe a galaxy / nebula / extended object. What do I do?

You'll need a little more information than just the magnitude if you want to observe an extended object.

For example, I can look up on [Simbad](https://simbad.u-strasbg.fr/simbad/) that M101 has V=7.86 mag. But what does this actually mean for an extended object? Typically, the magnitude represents the integrated surface brightness of the source - that is, how bright it would be if it was collapsed to the size of a point source.

This means that you can go the other way to estimate how long you need to observe an extended object. "But wait!" you may cry, "how can I get the size of my object?" Good question. Out of curiosity - have you made your finding chart yet??

In [ ]:
# code